In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class NN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(NN, self).__init__()
        self.layer1 = nn.Linear(input_size, 50)
        self.layer2 = nn.Linear(50, num_classes)
        
    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = self.layer2(x)
        return x

In [13]:
# Hyperparams
input_size = 784 #28*28
num_classes =10
learning_rate = 0.001
batch_size = 100
num_epochs = 10

In [5]:
# MNIST Dataset
# Data Loading
train_dataset = datasets.MNIST(root='Data/MNIST-Dataset', train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = datasets.MNIST(root='Data/MNIST-Dataset', train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [6]:
model = NN(input_size=input_size, num_classes=num_classes).to(device)
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [12]:
# for batch_idx, (data, targets) in enumerate(train_loader):
print(train_dataset[0])
print(len(train_dataset))

(tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000

In [21]:
for batch_idx, (data, targets) in enumerate(train_loader):
    print(batch_idx)
    print(data.shape)
    print(targets.shape)
    # print(data[0])
    print()
    break

0
torch.Size([64, 1, 28, 28])
torch.Size([64])



In [30]:
model.train()  # Setting model into training mode

for epoch in range(num_epochs):
    num_samples = 0
    num_correct = 0
    for batch_idx, (data, targets) in enumerate(train_loader):
        data = data.to(device=device)
        targets = targets.to(device=device)
        # Re-sizing data to feed NN
        data = data.reshape(data.shape[0], -1)
        
        # Output prediction
        predicted_output = model(data)
        
        # Calculate Accuracy
        _, prediction = predicted_output.max(1)
        num_samples += predicted_output.size(0)
        num_correct += (prediction==targets).sum()
        
        # calculate predicted output and loss
        loss = loss_function(predicted_output, targets)
        # Resetting gradients to zero and update weights based on gradients computed in loss.backward()
        optimizer.zero_grad()
        loss.backward()
        # Performing gradient descent
        optimizer.step()
        
        if ((batch_idx+1) % 200 == 0):
            print(f'Batch {batch_idx+1}: Accuracy: {num_correct / num_samples * 100.00}')
    print(f'Epoch {epoch+1} completed with Accuracy {num_correct / num_samples * 100.00}')
    print()
print()
print("Training Completed!")

        
   

Batch 200: Accuracy: 99.9453125
Batch 400: Accuracy: 99.91796875
Batch 600: Accuracy: 99.9140625
Batch 800: Accuracy: 99.90234375
Epoch 1 completed with Accuracy 99.89666748046875

Batch 200: Accuracy: 99.890625
Batch 400: Accuracy: 99.8984375
Batch 600: Accuracy: 99.89322662353516
Batch 800: Accuracy: 99.8359375
Epoch 2 completed with Accuracy 99.84333801269531

Batch 200: Accuracy: 99.8984375
Batch 400: Accuracy: 99.90234375
Batch 600: Accuracy: 99.90885162353516
Batch 800: Accuracy: 99.904296875
Epoch 3 completed with Accuracy 99.89500427246094

Batch 200: Accuracy: 99.8984375
Batch 400: Accuracy: 99.9375
Batch 600: Accuracy: 99.9453125
Batch 800: Accuracy: 99.94140625
Epoch 4 completed with Accuracy 99.92333221435547

Batch 200: Accuracy: 99.890625
Batch 400: Accuracy: 99.8828125
Batch 600: Accuracy: 99.89322662353516
Batch 800: Accuracy: 99.880859375
Epoch 5 completed with Accuracy 99.87000274658203

Batch 200: Accuracy: 99.90625
Batch 400: Accuracy: 99.9375
Batch 600: Accuracy: 9

In [38]:
# Model Testing
def check_accuracy(loader, model):
    model.eval() # Set model into evaluation mode!
    num_samples = 0
    num_correct = 0
    for data, targets in loader:
            data = data.to(device=device)
            targets = targets.to(device=device)
            data = data.reshape(data.shape[0], -1)
            predicted_output = model(data)

            # Calculate Accuracy
            _, prediction = predicted_output.max(1)
            num_samples += predicted_output.size(0)
            num_correct += (prediction==targets).sum()
    
    print(f'Correctly identified samples: {num_correct}')
    print(f'Total samples: {num_samples}')
    print(f'The Validation Accuracy is {num_correct / num_samples * 100.00:.2f}')
    model.train() # Setting model back to training data

In [39]:
print("Checking Accuracy on Training Data...")
check_accuracy(train_loader, model)
print()
print("Checking Accuracy on Testing Data...")
check_accuracy(test_loader, model)

Checking Accuracy on Training Data...
Correctly identified samples: 59958
Total samples: 60000
The Validation Accuracy is 99.93

Checking Accuracy on Testing Data...
Correctly identified samples: 9713
Total samples: 10000
The Validation Accuracy is 97.13
